In [60]:
pip install anytree

Note: you may need to restart the kernel to use updated packages.


In [61]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [62]:
pip install inflect

Note: you may need to restart the kernel to use updated packages.


# Importingnltk.download('punkt')
nltk.download('stopwords') Libraries

In [63]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kasiv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kasiv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [64]:
import re
from anytree import Node, RenderTree
import string
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
import json
import inflect
import csv
import sys

In [65]:
import inflect
p = inflect.engine()

In [66]:
custom_stop_words = ["Else","Facilities","Associated","less","service","Premium","Processing","Product","Provide","Includes","supply","Standard","new","class","group","offer","package","facility","accessory","part","customer","function","management","station","block","personal","material","machinary","storage","warehouse"]
custom_stop_words_singular=[]

In [67]:
 for word in custom_stop_words:
        ele = p.singular_noun(word)
        if type(ele) == str:
            custom_stop_words_singular.append(ele.lower())
        else:
            custom_stop_words_singular.append(word.lower())
            

In [68]:
stop_words= set(stopwords.words("english") + custom_stop_words_singular)

# Text Preprocessing

In [69]:
def text_preprocessing(data):
  data = data.encode('ascii', 'ignore').decode()
  data = re.sub(r'\'\w+', '', data)
  data = re.sub('[%s]' % re.escape(string.punctuation), ' ', data)
  data = re.sub(r'\w*\d+\w*', '', data)
  data = re.sub(r'\s{2,}', ' ', data)
  data = data.lower()
    
  clensed_data = ""
  for word in data.split(' '):
     singular = p.singular_noun(word)
     if type(singular) != str :
            singular = word
     if singular not in stop_words:        
            clensed_data += singular + " "
            
  return clensed_data.rstrip()

# Levels to load

In [70]:
titles=[["Segment Title","Segment"],["Family Title","Family"],["Class Title","Class"],["Commodity Title","Commodity"]]

# Key Generation

In [71]:
# we create the key representing the start char of every word in the sentence like if the sentence is like "can we create the key and find similar keys?" in the alphabet sequence array
# The advantage of this approach is that we can find the similar keys in short time
# The first_char_array_title will contains [c,w,t,k,a,f,s] unique values
# The key will be 10100100001000000011001000 because we assign 
# 
# for Char 'c' => arr[2]=1 ; Char 'w' => arr[22]=1 ; Char 't' => arr[19]=1 ; Char 'k' => arr[10]=1 ; Char 'a' => arr[0]=1 ; Char 'f' => arr[5]=1 ; Char 's' => arr[18]=1 ; 

def create_node_key(title):

    char_array = ['0']*26
    first_char_array_title =set(map(lambda x: x[0] ,filter(lambda x:x!="",title.strip().split(' '))))

    for start_char in first_char_array_title:
        index= ord(start_char.lower()) - ord('a')
        if  index >=0 and index<=25:
            char_array[index] = '1'

    key = "".join(char_array)
    return key

# We are finding the similar keys by making the key into binary format and simply using the bitwise and operator to see if the values !=0 which means the key1 and key2 are having similar keys
def cmp_key(key1,key2):
    return (eval('0b' + key1) & eval('0b' + key2) !=0)

# Extracting UNSPSC File and store the data in tree format using <b>AnyTree Library</b>

## Tree pointers

In [72]:
#Creating Root Node
root=Node("root")
nodes={}
nodes[root.name]=root

#Pointer to the levels
tree_pointer = {}
tree_pointer ["Segment Title"]={}
tree_pointer ["Family Title"]={}
tree_pointer ["Class Title"]={}
tree_pointer ["Commodity Title"]={}

## Creating Node

In [73]:
def create_Node(data,title):
  global tree_pointer
  parent_key=""
  if (data["key"].strip()==''):
      return

  node_key = create_node_key(data['title'])
  if node_key in tree_pointer[title]:
    parent_key = tree_pointer[title][node_key]

  else:
     tree_pointer[title][node_key] = Node(node_key,parent=nodes[title])
     parent_key =  tree_pointer[title][node_key]

  if not data["title"] in set(map(lambda child: child.name,parent_key.children)):   
      Node(data["title"],parent=parent_key,data=data)


## load UNSPSC File

In [74]:
%%time
with open('UNSPSC_English.csv',encoding="utf8")  as unspsc_file:
    [ next(unspsc_file) for _ in range(9)]   
    unspsc_reader = csv.DictReader(unspsc_file, delimiter=',')
    [ next(unspsc_reader) for _ in range(2)] 
    for title in titles:
      nodes[title[0]] = Node(title[0],parent=root)

    for line in unspsc_reader:
      for index in range(0,len(titles)):
        create_Node({"title" : text_preprocessing(line[titles[index][0]]),"key":line[titles[index][1]]},titles[index][0]) 


Wall time: 2min 8s


## Unique Values of titles in UNSPSC File

In [75]:

print("Segement Title Leaves Count: ",len(nodes["Segment Title"].leaves),"                 Segement Title Children Count: ",len(nodes["Segment Title"].children))

print("Family Title Leaves Count: ",len(nodes["Family Title"].leaves),"                  Family Title Children Count: ",len(nodes["Family Title"].children))

print("Class Title Leaves Count: ",len(nodes["Class Title"].leaves),"                  Class Title Children Count: ",len(nodes["Class Title"].children))

print("Commodity Title Leaves Count: ",len(nodes["Commodity Title"].leaves),"            Commodity Title Children Count: ",len(nodes["Commodity Title"].children))

Segement Title Leaves Count:  57                  Segement Title Children Count:  54
Family Title Leaves Count:  541                   Family Title Children Count:  363
Class Title Leaves Count:  7796                   Class Title Children Count:  2957
Commodity Title Leaves Count:  146404             Commodity Title Children Count:  25782


# Load Avalara File

In [76]:
df = pd.read_excel('Avalara_goods_and_services.xlsx', header=1)
data_array = np.array(df)
avalara_file = data_array[1706:2547]
avalara_file=avalara_file[:,0:3]

In [77]:
text_preprocessing_vf = np.vectorize(text_preprocessing)
create_node_key_vf = np.vectorize(create_node_key)
avalara_file = np.column_stack((avalara_file,text_preprocessing_vf(avalara_file[:,1])))

In [78]:
avalara_file = np.column_stack((avalara_file,create_node_key_vf(avalara_file[:,3])))

# Searching the tree based on the key

In [79]:
def search_tree(key,level):
    key_match = {}    
    for child in nodes[level].children:
          if(cmp_key(child.name,key)):
            key_match[child.name] = child.children
    
    return key_match

# LCS

In [80]:
def longestCommonSubsequence(text1, text2):

    text1 = text1.lower().split(" ")
    text2 = text2.lower().split(" ")
    
    if len(text1) > len(text2):
        text2,text1 = text1,text2  
    arr = [[0 for i in range(len(text2)+1)] for j in range(2)]
    
    b=1
    for i in range (1,len(text1)+1):    
        b= i & 1
        for j in range(1,len(text2)+1):
            if text1[i-1] == text2[j-1]:
                arr[1-b][j]= arr[b][j-1]+1
            else:
                arr[1-b][j] = max(arr[b][j],arr[1-b][j-1])
    
    return arr[1-b][-1]

# Main function for finding the match ratio

In [81]:
def try_function_parallel_all_levels(data,ans):
    levels = ["Commodity Title","Class Title","Family Title","Segment Title"]
    for index,level in enumerate(levels):
      for match_parent in (search_tree(data[4],level)).values():
        for match in match_parent:
            avalara_desc = data[3] 
            avalara_key = data[0] 
            if not isinstance(avalara_key,str):
                return
            unspsc_desc = match.data["title"]
            unspsc_key = match.data["key"]
            avalara_length = len(avalara_desc.split(" "))
            unspsc_length = len(unspsc_desc.split(" "))
            min_length = unspsc_length if (avalara_length > unspsc_length) else avalara_length
            max_length = unspsc_length if (avalara_length < unspsc_length) else avalara_length
            thresold = 0.5
            lcs = longestCommonSubsequence(unspsc_desc,avalara_desc)
            match_perc_min = lcs/min_length
            match_perc_max = lcs/max_length
            if (match_perc_min >= thresold):
                if avalara_desc in ans:
                  if level in ans[avalara_desc]:
                    ans[avalara_desc][level].append({"unspsc_text" : unspsc_desc,"unspsc_key":unspsc_key,"match_ratio_min" : match_perc_max,"match_ratio_max":match_perc_min })
                  else:
                    ans[avalara_desc][level] = []
                    ans[avalara_desc][level].append({"unspsc_text" : unspsc_desc,"unspsc_key":unspsc_key,"match_ratio_min" : match_perc_max,"match_ratio_max":match_perc_min })
                else:
                  ans[avalara_desc] ={"key":avalara_key}
                  ans[avalara_desc][level] = []
                  ans[avalara_desc][level].append({"unspsc_text" : unspsc_desc,"unspsc_key":unspsc_key,"match_ratio_min" : match_perc_max,"match_ratio_max":match_perc_min})

# Multiprocessing Finding match

In [82]:
%%time

ans = {}
total_length_avalara_file = len(avalara_file)

num_cores = multiprocessing.cpu_count()
Parallel(n_jobs=num_cores, verbose=10,require='sharedmem')(delayed(try_function_parallel_all_levels)(i,ans) for i in avalara_file)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:   23.9s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   40.8s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:   56.2s
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:  1.3min
[Parallel(n_jobs=16)]: Done  53 tasks      | elapsed:  1.6min
[Parallel(n_jobs=16)]: Done  66 tasks      | elapsed:  2.1min
[Parallel(n_jobs=16)]: Done  81 tasks      | elapsed:  2.4min
[Parallel(n_jobs=16)]: Done  96 tasks      | elapsed:  2.9min
[Parallel(n_jobs=16)]: Done 113 tasks      | elapsed:  3.5min
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:  4.1min
[Parallel(n_jobs=16)]: Done 149 tasks      | elapsed:  4.7min
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:  5.4min
[Parallel(n_jobs=16)]: Done 189 tasks      | elapsed:  5.9min
[Parallel(n_jobs=16)]: Done 210 tasks      | elapsed:  6.6min
[Parallel(n_jobs=16)]: Done 233 tasks      | elaps

Wall time: 28min 45s


[Parallel(n_jobs=16)]: Done 839 out of 839 | elapsed: 28.8min finished


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [1]:
ans["tattoo"]

NameError: name 'ans' is not defined

In [ ]:
test_Cases = ["Warehouse storage - storage of refrigerated goods","Word processing","Telephone support - hosted software - for business use","Settlement services - related to real estate","Title services - related to real estate","Limousine services","Taxidermy","Tattoo services","Snow removal services -performed on property grounds - excludes snow removal from atop structures","Candles and candle accessories","Office supplies","Pet - ear washes","School related products (b2b)-supplies-protractors","School related products (b2c)-supplies-compasses","Product / consumer electronics / portable electronics / personal devices / personal digital assistant (pda)","Baby supplies - strollers","Accounting-other","Cigarettes","Buttons","Cloud services - platform as a service (paas)","Watercraft (other)-repair","Limousine services","Security","Storage services","Background checks","Legal services - in-person services","Extermination","Daycare and babysitting services","Environmental consulting services"]

In [ ]:
test_Cases = ["Safe deposit box rentals"]

In [ ]:

levels = ["Commodity Title","Family Title","Class Title","Segment Title"]
level_out={}
for text in test_Cases:
    for level in levels:
        try:
            if text not in ans:
                ans[text] = {}
                ans[text][level]=(sorted(sorted(list(map(lambda x : {"data":x,"normalized_value":(1-x["match_ratio_min"])*(x["match_ratio_min"]-x["match_ratio_max"])} ,level_out[avalara_file[:,][avalara_file[:,1] == text][0][3]][level])),key=lambda x:x["normalized_value"]),key=lambda x:x["data"]["match_ratio_max"],reverse=True)[0]) 
            else:
                ans[text][level]=(sorted(sorted(list(map(lambda x : {"data":x,"normalized_value":(1-x["match_ratio_min"])*(x["match_ratio_min"]-x["match_ratio_max"])} ,level_out[avalara_file[:,][avalara_file[:,1] == text][0][3]][level])),key=lambda x:x["normalized_value"]),key=lambda x:x["data"]["match_ratio_max"],reverse=True)[0])
        except:
            continue



In [ ]:
level_out[avalara_file[:,][avalara_file[:,1] == "Tattoo services"][0][3]]

In [97]:
levels = ["Commodity Title","Family Title","Class Title","Segment Title"]
best_result = {}
for desc in ans:
    for level in levels:
        try:
            if desc not in best_result:
                best_result[desc] = {}
                best_result[desc][level]=(sorted(sorted(list(map(lambda x : {"data":x,"normalized_value":(1-x["match_ratio_max"])*(x["match_ratio_max"]-x["match_ratio_min"])} ,ans[desc][level])),key=lambda x:x["normalized_value"]),key=lambda x:x["data"]["match_ratio_min"],reverse=True)[0]) 
            else:
                best_result[desc][level]=(sorted(sorted(list(map(lambda x : {"data":x,"normalized_value":(1-x["match_ratio_max"])*(x["match_ratio_max"]-x["match_ratio_min"])} ,ans[desc][level])),key=lambda x:x["normalized_value"]),key=lambda x:x["data"]["match_ratio_min"],reverse=True)[0]) 
        except:
            continue
                

In [95]:
(sorted(sorted(list(map(lambda x : {"data":x,"normalized_value":(1-x["match_ratio_max"])*(x["match_ratio_max"]-x["match_ratio_min"])} ,ans["health care counter medicinal drug fact ingredient included deem medicine home use"]["Commodity Title"])),key=lambda x:x["normalized_value"]),key=lambda x:x["data"]["match_ratio_min"],reverse=True)[0]) 

{'data': {'unspsc_text': 'drug medicine production',
  'unspsc_key': '73101701',
  'match_ratio_min': 0.16666666666666666,
  'match_ratio_max': 0.6666666666666666},
 'normalized_value': 0.16666666666666669}

In [22]:
best_result["tattoo"]

{'Commodity Title': {'data': {'unspsc_text': 'tattoo',
   'unspsc_key': '53131631',
   'match_ratio_min': 1.0,
   'match_ratio_max': 1.0},
  'normalized_value': 0.0},
 'Class Title': {'data': {'unspsc_text': 'tattoo equipment',
   'unspsc_key': '53131800',
   'match_ratio_min': 0.5,
   'match_ratio_max': 1.0},
  'normalized_value': 0.0}}

In [ ]:
ans={}
try_function_parallel_all_levels(avalara_file[:,][avalara_file[:,1] == text][0],ans)
sorted(sorted(list(map(lambda x : {"data":x,"normalized_value":(1-x["match_ratio_min"])*(x["match_ratio_min"]-x["match_ratio_max"])} ,ans[avalara_file[:,][avalara_file[:,1] == text][0][3]]["Commodity Title"])),key=lambda x:x["normalized_value"]),key=lambda x:x["data"]["match_ratio_max"],reverse=True)[0]

In [ ]:
avalara_file[:,][avalara_file[:,1] == text][0][3]

In [39]:
# Save the DataFrame to a file
with open('final_result.json', 'w') as f:
  f.write(json.dumps(final_result))

In [2]:
import json
 
# Opening JSON file
f = open('best_result.json')
 
# returns JSON object as
# a dictionary
best_result = json.load(f)

In [ ]:
best_result

In [27]:
for i in best_result["home fragrance"].items():
    min_val = 
    print(i)

('Commodity Title', {'data': {'unspsc_text': 'home treatment', 'unspsc_key': '85904211', 'match_ratio_min': 0.5, 'match_ratio_max': 0.5}, 'normalized_value': 0.0})
('Class Title', {'data': {'unspsc_text': 'home interior design instructional', 'unspsc_key': '60105500', 'match_ratio_min': 0.25, 'match_ratio_max': 0.5}, 'normalized_value': 0.125})


In [36]:
final_result ={}
for avalara_desc in best_result: 
    final_result[avalara_desc] = sorted(best_result[avalara_desc].items(), key=lambda k: k[1]['normalized_value'])[0]

In [38]:
best_result['health care counter medicinal first aid kit']

{'Commodity Title': {'data': {'unspsc_text': 'mobile medical first aid kit',
   'unspsc_key': '42172001',
   'match_ratio_min': 0.42857142857142855,
   'match_ratio_max': 0.6},
  'normalized_value': 0.06857142857142857},
 'Family Title': {'data': {'unspsc_text': 'care',
   'unspsc_key': '53130000',
   'match_ratio_min': 0.14285714285714285,
   'match_ratio_max': 1.0},
  'normalized_value': 0.0},
 'Class Title': {'data': {'unspsc_text': 'first aid room furnishing',
   'unspsc_key': '56121200',
   'match_ratio_min': 0.2857142857142857,
   'match_ratio_max': 0.5},
  'normalized_value': 0.10714285714285715}}

In [ ]:
testcases = ["SL090200","SW117218","SW090000","SW052401","ST157135","ST157137","ST096738","ST037114","ST027106","SS197074","PH060754","PO100000","PP052108", "PS050124", "PS060106","PB0010502","PI041079","SA010300", "PS160747", "PS080101","SW056000","SW040700","ST096738","SS040000","SS130000","SS046330","SL060100","SE090000","SC223100","SC163000"]

In [37]:
final_result

{'health care counter medicinal drug fact ingredient included deem medicine home use': ('Family Title',
  {'data': {'unspsc_text': 'care',
    'unspsc_key': '53130000',
    'match_ratio_min': 0.08333333333333333,
    'match_ratio_max': 1.0},
   'normalized_value': 0.0}),
 'health care counter medicinal pain reliever': ('Family Title',
  {'data': {'unspsc_text': 'care',
    'unspsc_key': '53130000',
    'match_ratio_min': 0.16666666666666666,
    'match_ratio_max': 1.0},
   'normalized_value': 0.0}),
 'health care counter medicinal first aid kit': ('Family Title',
  {'data': {'unspsc_text': 'care',
    'unspsc_key': '53130000',
    'match_ratio_min': 0.14285714285714285,
    'match_ratio_max': 1.0},
   'normalized_value': 0.0}),
 'counter medicinal foot pad insole': ('Commodity Title',
  {'data': {'unspsc_text': 'counter',
    'unspsc_key': '41111901',
    'match_ratio_min': 0.2,
    'match_ratio_max': 1.0},
   'normalized_value': 0.0}),
 'health care counter medicinal syringe disposabl

In [ ]:
text = "Warehouse storage - storage of refrigerated goods"
print(text_preprocessing(text))
sorted(list(map(lambda x : {"data":x,"normalized_value":x["match_ratio_min"]-x["match_ratio_max"]} ,data[text_preprocessing(text)]["Commodity Title"])),key=lambda x:x["normalized_value"])

In [ ]:
text_preprocessing("Warehouse storage - storage of refrigerated goods")

In [ ]:
list(filter(lambda x : x["data"]["title"]=="cat",nodes["Commodity Title"].leaves))

In [107]:
from anytree.exporter import DotExporter
DotExporter(nodes["Segment Title"]).to_picture("Segment_title.png")

FileNotFoundError: [WinError 2] The system cannot find the file specified